# Emerging Technologies Problems

In [29]:
# Libraries used in this notebook:
import random
import numpy as np
import itertools as it

## Problem 1: Generating Random Boolean Functions

The Deutsch–Jozsa algorithm requires functions that accept a fixed number of Boolean inputs and return a single Boolean output. Each function must be either:
- **Constant**: Always returns 0 or always returns 1
- **Balanced**: Returns 1 for exactly half of the possible input combinations

For 4 Boolean inputs, there are $2^4 = 16$ possible input combinations, meaning:
- 2 constant functions (all 0s or all 1s)
- $\binom{16}{8} = 12870$ balanced functions (exactly 8 ones and 8 zeros)

### Approach

Following the examples from [IBM Quantum Learning](https://quantum.cloud.ibm.com/learning/en/courses/basics-of-quantum-information/single-systems/classical-information#deterministic-operations), we use:
1. **Random tuple generation** to create the function's output pattern
2. **Closures** to encapsulate the function's truth table ([Python Closures on Real Python](https://realpython.com/python-closures/))
3. **Lambda functions** for anonymous function creation ([How to Use Python Lambda Functions on Real Python](https://realpython.com/python-lambda/))
4. **Dictionary-based lookup** for efficient function evaluation

The implementation randomly selects between constant and balanced types with 50/50 probability, then returns a callable function that performs lookups on its internal truth table.

In [30]:
def random_constant_balanced():
    """
    Returns a randomly chosen function that accepts 4 Boolean arguments and 
    returns a single Boolean output.
    
    The function is guaranteed to be either:
    - Constant: always returns 0 or always returns 1
    - Balanced: returns 1 for exactly 8 of the 16 possible input combinations
    
    Returns:
        A callable function that takes a 4-tuple of bits (0 or 1) and returns 0 or 1.
    """
    # Generate all 16 possible 4-bit input combinations.
    # e.g., (0,0,0,0), (0,0,0,1), ..., (1,1,1,1)
    all_inputs = list(it.product([0, 1], repeat=4))
    
    # Randomly choose function type - 50/50 probability.
    ftype = random.choice(['constant', 'balanced'])
    
    if ftype == 'constant':
        # Choose value (0 or 1) to return for all inputs.
        output_value = random.choice([0, 1])
        # Map all 16 inputs to the same output.
        truth_table = {inp: output_value for inp in all_inputs}
    else:  # balanced
        # Randomly select 8 of 16 inputs to output 1, rest output 0.
        ones_inputs = random.sample(all_inputs, 8)
        # Build truth table: 8 inputs -> 1, other 8 inputs -> 0.
        truth_table = {inp: (1 if inp in ones_inputs else 0) for inp in all_inputs}
    
    # Return a lambda that looks up the input in the truth table.
    return lambda x: truth_table[x]

## Problem 2: Classical Testing for Function Type

Given a black-box Boolean function, we need to determine whether it is **constant** or **balanced**. The classical approach requires testing the function with multiple inputs and analyzing the outputs.

### Classical Testing Requirements

For a function with 4 Boolean inputs ($n = 4$ bits):
- **Total possible inputs**: $2^4 = 16$ combinations
- **Worst-case queries needed**: In the worst case, we must test **9 inputs** to determine with certainty whether the function is constant or balanced
  - If the first 8 outputs are all the same, we cannot be sure if it's constant (all same) or balanced (the 9th might differ)
  - The 9th query confirms the pattern

### Implementation Approach

Our classical implementation:
- Tests the function on all 16 possible input combinations
- Counts the number of 1s in the outputs
- Classifies based on the count:
  - **0 or 16 ones** → Constant function
  - **Exactly 8 ones** → Balanced function

In [31]:
def determine_constant_balanced():
    """
    Determines whether the function returned by random_constant_balanced() is 
    constant or balanced by querying it with different inputs.
    
    Returns:
        A string "constant" if the function is constant, or "balanced" if it is balanced.
    """
    # Test the function.
    f = random_constant_balanced()

    # Test all 16 inputs to verify constant or balanced property.
    all_outputs = [f(combo) for combo in it.product([0, 1], repeat=4)]
    ones_count = sum(all_outputs)

    print(f"\nTotal 1s out of 16 inputs: {ones_count}")
    if ones_count == 0 or ones_count == 16:
        print("Function is CONSTANT")
    elif ones_count == 8:
        print("Function is BALANCED")
    else:
        print("ERROR: Function is neither constant nor balanced!")
    

In [32]:
determine_constant_balanced()


Total 1s out of 16 inputs: 16
Function is CONSTANT


## Problem 3: Quantum Oracles

### Overview: The Four Boolean Functions

For Deutsch's algorithm with a single Boolean input, there are exactly **4 possible functions** $f: \{0,1\} \rightarrow \{0,1\}$:

**Constant Functions** (output independent of input):
1. **f(x) = 0**: Always returns 0
2. **f(x) = 1**: Always returns 1

**Balanced Functions** (output equals 0 for half the inputs, 1 for the other half):
3. **f(x) = x**: Identity function
4. **f(x) = NOT x**: Negation function

A quantum oracle $U_f$ implements each function as: $U_f|x\rangle|y\rangle = |x\rangle|y \oplus f(x)\rangle$


### Oracle 1: Constant-0 Function ($f(x) = 0$)

**Function definition:**
- $f(0) = 0$
- $f(1) = 0$

**Oracle behavior:**
- $U_f|x\rangle|y\rangle = |x\rangle|y \oplus 0\rangle = |x\rangle|y\rangle$


In [ ]:
def constant_0():
    return 0

### Oracle 2: Constant-1 Function ($f(x) = 1$)

**Function definition:**
- $f(0) = 1$
- $f(1) = 1$

**Oracle behavior:**
- $U_f|x\rangle|y\rangle = |x\rangle|y \oplus 1\rangle$


In [ ]:
def constant_1():
    return 1    

### Oracle 3: Balanced Identity Function ($f(x) = x$)

**Function definition:**
- $f(0) = 0$
- $f(1) = 1$

**Oracle behavior:**
- When $x = 0$: $U_f|0\rangle|y\rangle = |0\rangle|y \oplus 0\rangle = |0\rangle|y\rangle$
- When $x = 1$: $U_f|1\rangle|y\rangle = |1\rangle|y \oplus 1\rangle$


In [ ]:
def balanced_indentity():
    return 0

### Oracle 4: Balanced Negation Function ($f(x) = \text{NOT } x$)

**Function definition:**
- $f(0) = 1$
- $f(1) = 0$

**Oracle behavior:**
- When $x = 0$: $U_f|0\rangle|y\rangle = |0\rangle|y \oplus 1\rangle$
- When $x = 1$: $U_f|1\rangle|y\rangle = |1\rangle|y \oplus 0\rangle = |1\rangle|y\rangle$


In [ ]:
def balanced_negation():
    return 0

## Problem 4: Deutsch's Algorithm with Qiskit

## Problem 5: Scaling to the Deutsch–Jozsa Algorithm